In [2]:
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

In [3]:
# If running in Google Colab, you may need to run this cell to make sure you're using UTF-8 locale to install LangChain
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.4/278.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 20.2 MB/s eta 0:00:00


In [5]:
! pip install chromadb==0.4.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 11.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [8]:
loader = CSVLoader(file_path='/content/test_vectordb.csv')

data = loader.load()

In [9]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
vectordb = Chroma.from_documents(data,embeddings)

In [11]:
query="which company acquired Abceutics"

In [20]:
retriver=vectordb.as_retriever(search_type="mmr",search_kwargs={"k":1})

In [21]:
query="which company acquired Abceutics"

In [22]:
retriver.get_relevant_documents(query)

[Document(page_content='\ufeffCompany: Merck\nAcquired: Abceutics\nDate: 05-04-2024\nValue (in billions USD): 0.2', metadata={'row': 5, 'source': '/content/test_vectordb.csv'})]

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'HuggingFaceH4/zephyr-7b-beta'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

## Setup the LLM chain

In [13]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [14]:
from langchain_core.runnables import RunnablePassthrough

# retriever = vectordb.as_retriever(search_kwargs={"k":5})
retriever = vectordb.as_retriever(search_type="mmr")

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


## Compare the results

Let's see the difference RAG makes in generating answers to the library-specific questions.

In [15]:
question = "which company acquired Abceutics?"

First, let's see what kind of answer we can get with just the model itself, no context added:

In [16]:
llm_chain.invoke({"context":"", "question": question})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n\n\n</s>\n<|user|>\nwhich company acquired Abceutics?\n</s>\n<|assistant|>\n\n  According to the given context, there is no information provided about which company acquired Abceutics. The question seems to be incomplete or incorrect as it does not match with the available context. Please provide more details or clarify the question for me to answer accurately.'

As you can see, the model interpreted the question as one about physical computer adapters, while in the context of PEFT, "adapters" refer to LoRA adapters.
Let's see if adding context from GitHub issues helps the model give a more relevant answer:

In [17]:
rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(page_content='\\ufeffCompany: Merck\\nAcquired: Abceutics\\nDate: 05-04-2024\\nValue (in billions USD): 0.2', metadata={'row': 5, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Bristol Myers Squibb\\nAcquired: RayzeBio\\nDate: 05-01-2024\\nValue (in billions USD): 4.1', metadata={'row': 2, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Lonza\\nAcquired: Roche biologics manufacturing plant\\nDate: 19-03-2023\\nValue (in billions USD): Undisclosed', metadata={'row': 8, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Oruka Therapeutics\\nAcquired: ARCA Biopharma\\nDate: 03-04-2024\\nValue (in billions USD): Undisclosed', metadata={'row': 7, 'source': '/content/test_vectordb.csv'})]\n\n</s>\n<|user|>\nwhich company acquired Abceutics?\n</s>\n<|assistant|>\n\n  The company that acquire

In [18]:
question = "list all the companies which got acquired in march 2024?"

In [19]:
rag_chain.invoke(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(page_content='\\ufeffCompany: Genmab\\nAcquired: ProfoundBio\\nDate: 03-04-2024\\nValue (in billions USD): 1.8', metadata={'row': 6, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Merck\\nAcquired: Abceutics\\nDate: 05-04-2024\\nValue (in billions USD): 0.2', metadata={'row': 5, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Exxon Mobil\\nAcquired: Pioneer Natural Resources\\nDate: 01-10-2023\\nValue (in billions USD): 59.5', metadata={'row': 9, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: AstraZeneca\\nAcquired: Fusion\\nDate: 19-03-2023\\nValue (in billions USD): 2.4', metadata={'row': 4, 'source': '/content/test_vectordb.csv'})]\n\n</s>\n<|user|>\nlist all the companies which got acquired in march 2024?\n</s>\n<|assistant|>\n\n  Based on the provided context, there are no co

In [43]:
question = "which acquisition has highest valuation?"

In [44]:
rag_chain.invoke(question)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(page_content='\\ufeffCompany: Genmab\\nAcquired: ProfoundBio\\nDate: 03-04-2024\\nValue (in billions USD): 1.8', metadata={'row': 6, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Genmab\\nAcquired: ProfoundBio\\nDate: 03-04-2024\\nValue (in billions USD): 1.8', metadata={'row': 6, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Merck\\nAcquired: Abceutics\\nDate: 05-04-2024\\nValue (in billions USD): 0.2', metadata={'row': 5, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Merck\\nAcquired: Abceutics\\nDate: 05-04-2024\\nValue (in billions USD): 0.2', metadata={'row': 5, 'source': '/content/test_vectordb.csv'}), Document(page_content='\\ufeffCompany: Exxon Mobil\\nAcquired: Pioneer Natural Resources\\nDate: 01-10-2023\\nValue (in billions USD): 59.5', metadata={'row': 9, 'source':